In [7]:
#Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score


In [8]:
#asssign csv file to variable
data = pd.read_csv("diamond_data_merged_with_other_variables.csv")
data

,date,diamond price,inflation rate,interest rate,fed rate,gold price
0,2021-06-04,10385.16,2.40,-0.84,0.06,1890.97
1,2021-06-03,10385.16,2.42,-0.79,0.06,1871.36
2,2021-06-02,10378.43,2.44,-0.85,0.06,1907.75
3,2021-06-01,10376.10,2.46,-0.84,0.06,1900.49
4,2021-05-31,10368.67,2.46,-0.84,0.05,1905.74
...,...,...,...,...,...,...
1129,2018-05-02,10047.16,2.17,0.80,1.70,1302.13
1130,2018-05-01,10050.07,2.16,0.81,1.70,1300.58
1131,2018-04-30,10038.62,2.17,0.78,1.69,1315.38
1132,2018-04-29,10032.95,2.17,0.78,1.70,1322.80


In [10]:
#check if there are null values
data.isnull().sum()


date              0
diamond price     0
inflation rate    0
interest rate     0
fed rate          0
gold price        0
dtype: int64

In [13]:
#remove date column 
data2 = data.iloc[:,1:5]

In [28]:
#function to check for oob score with random forest
def evaluate(X,y,n_estimators=50):
    rf = RandomForestRegressor(n_estimators=n_estimators,n_jobs=-1,oob_score=True)
    rf.fit(X,y)
    oob = rf.oob_score_
    #n = rfnnodes(rf)
    #h = np.median(rfmaxdepths(rf))
    print(f"OOB R^2 is {oob:.5f} ")
    return rf, oob

In [22]:
#seperate non target features from target feature
basefeatures = ['diamond price','inflation rate', 'interest rate', 'fed rate']


In [25]:
X = data[basefeatures]
y = data['gold price']

In [29]:
# call the function above with target variable(y) and non target variable(X)
rf,oob_baseline_initial =evaluate(X,y,n_estimators=100)

OOB R^2 is 0.99549 


In [32]:
#using a different n estimator to check for any significant change
rf,oob_baseline_initial =evaluate(X,y,n_estimators=300)

OOB R^2 is 0.99555 


R squared is below 1 which is pretty decent.Next we want to test and predict 


In [36]:
# Using Skicit-learn to split data into training and testing sets
train_features, test_features, train_target, test_target = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [37]:
print('Training Features Shape:', train_features.shape)
print('Training Target Shape:', train_target.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Target Shape:', test_target.shape)

Training Features Shape: (850, 4)
Training Target Shape: (850,)
Testing Features Shape: (284, 4)
Testing Target Shape: (284,)


In [53]:
# train model
rf = RandomForestRegressor(n_estimators=150, random_state= 50)
rf.fit(train_features,train_target)

RandomForestRegressor(n_estimators=150, random_state=50)

In [54]:
#make prediction
predictions = rf.predict(test_features)

#calculate the absolute errors
errors = abs(predictions - test_target)

# Mean Absolute error 
print('Mean Absolute Error:', round(np.mean(errors),2))

Mean Absolute Error: 13.34


In [55]:
#calculate mean absolute percentage error(mape)
mape = 100*(errors/test_target)

#calculate and display accuracy
accuracy= 100-np.mean(mape)

print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 99.16 %.


In [58]:
data[0:1]

,date,diamond price,inflation rate,interest rate,fed rate,gold price
0,2021-06-04,10385.16,2.4,-0.84,0.06,1890.97


In [61]:
#model.predict(pd.DataFrame([[10385.16,2.4,-0.84,0.06]]))